In [2]:
from tkinter import *
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import ttkbootstrap as tb
from ttkbootstrap.constants import *

# Initialize main window
root = tb.Window(themename="superhero")  # Modern theme
root.title("Image Steganography")
root.geometry("900x700")
root.resizable(False, False)
root.configure(bg="#1E1E1E")  # Dark background
selected_image = None  # Store selected image

# Function to encode message in image
def encode_message():
    global selected_image
    file_path = filedialog.askopenfilename(filetypes=[("PNG Images", "*.png")])
    if not file_path:
        return
    
    img = Image.open(file_path)
    selected_image = img
    display_image(img)

    message = text_entry.get("1.0", END).strip()
    
    if not message:
        messagebox.showerror("Error", "Message cannot be empty!")
        return
    
    encoded_img = img.copy()
    message += "EOF"  # End marker
    pixels = encoded_img.load()
    
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    
    if len(binary_message) > img.width * img.height:
        messagebox.showerror("Error", "Message too long for this image.")
        return
    
    data_index = 0
    for y in range(img.height):
        for x in range(img.width):
            r, g, b = pixels[x, y]
            if data_index < len(binary_message):
                new_r = (r & 0xFE) | int(binary_message[data_index])  # Modify LSB of Red channel
                pixels[x, y] = (new_r, g, b)
                data_index += 1
            if data_index >= len(binary_message):
                break
        if data_index >= len(binary_message):
            break
    
    save_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG Images", "*.png")])
    if save_path:
        encoded_img.save(save_path)
        messagebox.showinfo("Success", "Message encoded and image saved successfully!")

# Function to decode message from image
def decode_message():
    global selected_image
    file_path = filedialog.askopenfilename(filetypes=[("PNG Images", "*.png")])
    if not file_path:
        return
    
    img = Image.open(file_path)
    selected_image = img
    display_image(img)
    pixels = img.load()
    
    binary_message = ""
    for y in range(img.height):
        for x in range(img.width):
            r, _, _ = pixels[x, y]
            binary_message += str(r & 1)  # Extract LSB of Red channel
    
    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if message.endswith("EOF"):
            message = message[:-3]
            break
        message += char
    
    text_entry.delete("1.0", END)
    text_entry.insert(END, message)
    messagebox.showinfo("Decoded Message", f"Message extracted: \n\n{message}")

# Function to display the selected image
def display_image(img):
    img = img.resize((350, 350))
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

# UI Design
frame = tb.Frame(root, bootstyle="dark", padding=20)
frame.pack(pady=20, padx=20, fill=BOTH, expand=True)

header = tb.Label(frame, text="Image Steganography", font=("Arial", 24, "bold"), bootstyle="primary")
header.pack(pady=20)

image_label = tb.Label(frame, bootstyle="secondary")  # Placeholder for selected image
image_label.pack(pady=15)

input_frame = tb.Frame(frame, bootstyle="dark")
input_frame.pack(pady=10, fill=X)

encode_button = tb.Button(input_frame, text="🔐 Encode", command=encode_message, bootstyle="success-outline", padding=10)
encode_button.pack(side=LEFT, padx=10)

text_entry = Text(input_frame, height=5, width=50, font=("Arial", 12), bg="#2E2E2E", fg="white", insertbackground="cyan", border=2, relief=FLAT)
text_entry.pack(side=LEFT, padx=10, expand=True)


decode_button = tb.Button(input_frame, text="🔓 Decode", command=decode_message, bootstyle="info-outline", padding=10)
decode_button.pack(side=LEFT, padx=10)

root.mainloop()
